# Distance Classes to Nearest Soil Samples - SOC

Alessandro Samuel-Rosa, Taciara Zborowski Horst

2025-07-07

This script calculates the distance from each pixel of a reference map to the nearest soil sample point used for soil organic carbon stock modeling across the Brazilian biomes. It is designed to produce a map that visualizes the spatial representativeness of this specific sampling network.

The core calculation uses the efficient `ee.Image.fastDistanceTransform` method. The resulting distance-in-pixels is converted to a precise distance-in-meters using `ee.Image.pixelArea().sqrt()`, which accounts for map projection distortions.

Finally, the continuous distance values are reclassified into six distinct classes (e.g., 0-50 m, 50-250 m, up to >10,000 m). The output is a single-band, 8-bit integer image where each pixel value from 1 to 6 corresponds to one of these distance classes.

License: MIT

In [1]:
# --- 1. IMPORT LIBRARIES & INITIALIZE ---
import ee
import geemap

# Authenticate and initialize the Earth Engine project.
# ee.Authenticate() # Run once in your environment if needed.
ee.Initialize(project='mapbiomas-solos-workspace')

In [2]:
# --- 2. DEFINE ASSETS ---
# Soil sampling points.
samplingPoints = ee.FeatureCollection('projects/mapbiomas-workspace/SOLOS/AMOSTRAS/ORIGINAIS/2025-06-21-organic-carbon-stock-gram-per-square-meter')

# Reference soil property map.
# CORRECTED LINE: Select a valid band name from your image.
soilPropertyMap = ee.Image('projects/mapbiomas-workspace/SOLOS/PRODUTOS_C02/c02v2/mapbiomas_soil_collection2_v2_clay').select('clay_000_010cm')

# Area of Interest (AOI).
aoi = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/biomas_IBGE_250mil')

In [3]:
# --- 3. PROCESS DATA: CALCULATE DISTANCE CLASSES ---
# Create a base grid from the soil map for perfect alignment.
sourceImage = soilPropertyMap.multiply(0).paint(samplingPoints, 1)

# Apply the Fast Distance Transform (FDT).
fdt = sourceImage.fastDistanceTransform(512, 'pixels', 'squared_euclidean')

# Convert to a precise distance in meters using pixelArea().
distanceInPixels = fdt.select('distance').sqrt()
pixelSize = ee.Image.pixelArea().sqrt()
distanceInMeters = distanceInPixels.multiply(pixelSize)

# Reclassify the continuous distance into 6 discrete, meaningful classes.
distanceClasses = ee.Image(6) \
  .where(distanceInMeters.lte(10000), 5) \
  .where(distanceInMeters.lte(5000), 4) \
  .where(distanceInMeters.lte(1000), 3) \
  .where(distanceInMeters.lte(250), 2) \
  .where(distanceInMeters.lte(50), 1) \
  .toUint8()

# Clip the final raster to the boundary of the AOI.
finalClasses = distanceClasses.clip(aoi)

In [4]:
# --- 4. VISUALIZE THE RESULT ---
# Create an interactive map object.
Map = geemap.Map()
Map.add_basemap('SATELLITE')

# Define visualization parameters for the 6 classes.
visParams = {
  'min': 1,
  'max': 6,
  'palette': ["#4545FD", '#00A0FF', '#00FF00', '#FFFF00', '#FFA500', "#FC4242"]
}

# Add the final layer to the map and center the view.
Map.addLayer(finalClasses, visParams, 'Geostatistical Distance Classes')
Map.centerObject(aoi, 4)

# Display the interactive map in your notebook.
Map


Map(center=[-10.62046648393736, -53.18363392405513], controls=(WidgetControl(options=['position', 'transparent…